In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


df = pd.read_csv('20250623_161915_web_server_logs_2.csv')

In [2]:
# 파생 변수 생성
df['hour'] = pd.to_datetime(df['timestamp']).dt.hour

# status_code로부터 is_error, label을 새로 생성
df['is_error'] = (df['status_code'] >= 400).astype(int)
df['label'] = df['is_error']  # label도 따로 명시적으로 생성

# 사용할 피처 선택
features = ['hour', 'ip', 'method', 'status_code', 'size']

# 범주형 변수 인코딩
for col in ['ip', 'method']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 수치형 변수 스케일 조정
num_cols = ['hour', 'status_code', 'size']
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# 특성/라벨 분리
X = df[features]
y = df['label']

In [3]:
# 학습/테스트 데이터 분리 8:2 비율
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [4]:
# 7. 모델 학습 및 예측
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [5]:
# 8. 성능 평가
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       213
           1     1.0000    1.0000    1.0000        87

    accuracy                         1.0000       300
   macro avg     1.0000    1.0000    1.0000       300
weighted avg     1.0000    1.0000    1.0000       300



In [6]:
# hour, is_error 생성 확인
print(df[['timestamp', 'hour', 'status_code', 'is_error']].head(10))

             timestamp      hour  status_code  is_error
0  2024-11-30 09:26:01 -0.345817     0.202251         0
1  2024-11-30 12:00:34  0.084423    -0.729286         0
2  2024-12-01 00:22:12 -1.636535    -0.729286         0
3  2024-11-30 19:33:26  1.088314    -0.729286         0
4  2024-11-30 05:37:26 -0.919469    -0.729286         0
5  2024-11-30 10:02:02 -0.202404    -0.720062         0
6  2024-11-30 06:13:50 -0.776056     2.065324         1
7  2024-11-30 11:53:40 -0.058991     1.152234         1
8  2024-12-01 00:24:57 -1.636535    -0.720062         0
9  2024-11-30 04:37:41 -1.062883    -0.729286         0


# 웹 서버 로그 기반 악성 요청 분류 결과 요약

## 주요 내용
- **피처**: hour, ip, method, status_code, size (인코딩/스케일링 적용)
- **타깃(label)**: status_code 400 이상이면 1, 아니면 0으로 직접 생성

## 분류 성능

| label | precision | recall | f1-score | support |
|-------|-----------|--------|----------|---------|
| 0     | 1.00      | 1.00   | 1.00     | 213     |
| 1     | 1.00      | 1.00   | 1.00     | 87      |
| 전체  |           |        | **1.00** | 300     |

## 요약
- status_code가 label 기준이라, 모델이 정답을 100% 맞춤